## Contexto
BASE 01-A base representa internações hospitalares, com informações clínicas,
assistenciais e administrativas por evento de internação.

BASE 02- A base representa custos de exames, internações e procedimentos clínicos.

MERGE- As bases estão em um modelo de cardinalidade 1:N (uma internação pode possuir múltiplos itens associados), através da primekey {'senha_internacao'}.

## Objetivo do EDA – Base 01
Avaliar a qualidade, consistência e estrutura dos dados das bases.


Este EDA realizara cruzamentos de fontes externas https://github.com/cleytonferrari/CidDataSus/blob/master/CIDImport/Repositorio/Resources/CID-10-GRUPOS.CSV disponibilizado neste link.

# Setup e carregamento dos dados

In [3]:
# =========================
# Setup e configurações
# =========================
import os
os.chdir(r'C:\Projetos\case-ami-saude')

import pandas as pd
import numpy as np
import re
from scipy import stats

from src.utils import pipeline_universal_limpeza , cid_formato_valido , cid_compativel_especialidade
from src.utils import OutlierConfig, find_outliers, filter_outlier_rows, plot_outlier_boxplots, boxplot_iqr_todas_colunas

# EDA - BASE 01 | Internações Hospitalares

## Carregamento dos dados

In [4]:
os.getcwd()

'C:\\Projetos\\case-ami-saude'

In [5]:
# =========================
# Carregamento da Base 01
# =========================
path_base_01 = "C:\\Projetos\\case-ami-saude\\data\\raw\\Base 1 – Internações Hospitalares.csv"
df_base_01 = pd.read_csv(path_base_01)

df_base_01.shape


(20000, 36)

A base possui 20000 registros e 36 colunas.


## Entendimento estrutural da base

In [6]:
df_base_01.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   senha_internacao              20000 non-null  object 
 1   beneficiario_id               20000 non-null  object 
 2   numero_carteirinha            20000 non-null  object 
 3   nome_beneficiario             20000 non-null  object 
 4   data_nascimento               20000 non-null  object 
 5   idade                         20000 non-null  int64  
 6   sexo                          20000 non-null  object 
 7   uf                            20000 non-null  object 
 8   municipio                     20000 non-null  object 
 9   hospital_id                   20000 non-null  object 
 10  hospital_nome                 20000 non-null  object 
 11  perfil_hospital               20000 non-null  object 
 12  tipo_plano                    20000 non-null  object 
 13  s

In [7]:
df_base_01.describe(include="all")


,senha_internacao,beneficiario_id,numero_carteirinha,nome_beneficiario,data_nascimento,idade,sexo,uf,municipio,hospital_id,...,uti_flag,suporte_ventilatorio_flag,hemodialise_flag,tempo_autorizacao_horas,auditoria_responsavel,empresa_auditoria,status_regulacao,glosa_flag,valor_total_conta,valor_pago
count,20000,20000,20000,20000,20000,20000.00000,20000,20000,20000,20000,...,20000,20000,20000,20000.000000,20000,20000,20000,20000,20000.000000,20000.000000
unique,20000,20000,19777,3517,15156,NaN,2,18,52,80,...,2,2,2,NaN,2,11,4,2,NaN,NaN
top,SI20250019984,SI20250019984,CAR629618,F.C.C.,2019-09-22 00:00:00,NaN,F,ES,Brasília,H043,...,Não,Não,Não,NaN,Terceirizada,Empresa Terceira 1,Autorizada,Não,NaN,NaN
freq,1,1,3,69,5,NaN,11588,1914,865,310,...,16092,15400,18200,NaN,18200,7280,17233,14032,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,47.47160,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,50.434150,NaN,NaN,NaN,NaN,52200.338077,50799.017925
std,NaN,NaN,NaN,NaN,NaN,26.87835,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,82.653308,NaN,NaN,NaN,NaN,41977.555494,40979.501923
min,NaN,NaN,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,0.810000,0.810000
25%,NaN,NaN,NaN,NaN,NaN,27.00000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,14.000000,NaN,NaN,NaN,NaN,24537.535000,23820.372500
50%,NaN,NaN,NaN,NaN,NaN,46.00000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,28.000000,NaN,NaN,NaN,NaN,40438.015000,39346.245000
75%,NaN,NaN,NaN,NaN,NaN,68.00000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,41.000000,NaN,NaN,NaN,NaN,66653.667500,64916.427500


### Conclusão – Estrutura

A base apresenta estrutura compatível com granularidade por internação,
com colunas clínicas, administrativas e categóricas.
Há chave primária explícita como a senha de internação.


## Qualidade básica dos dados

In [8]:
# Valores ausentes
df_base_01.isna().mean().sort_values(ascending=False) * 100


cid_secundario                  55.07
motivo_alta                      6.00
data_alta                        6.00
senha_internacao                 0.00
nome_beneficiario                0.00
numero_carteirinha               0.00
beneficiario_id                  0.00
data_nascimento                  0.00
municipio                        0.00
hospital_id                      0.00
sexo                             0.00
idade                            0.00
perfil_hospital                  0.00
hospital_nome                    0.00
tipo_plano                       0.00
segmentacao_plano                0.00
data_solicitacao_autorizacao     0.00
acomodacao                       0.00
data_admissao                    0.00
uf                               0.00
data_autorizacao_senha           0.00
carater_internacao               0.00
especialidade_responsavel        0.00
tipo_internacao                  0.00
cid_principal                    0.00
complexidade                     0.00
uti_flag    

In [9]:
# Valores duplicados
df_base_01.duplicated().sum()


np.int64(0)

Conclusão : Não foram encontrados valores duplicados.

In [10]:
# Dominio de valores categoricos
for col in ["tipo_internacao", "carater_internacao", "especialidade_responsavel"]:
    print(col)
    display(df_base_01[col].value_counts(dropna=False, normalize=True) * 100)


tipo_internacao


tipo_internacao
Clínica         48.220
Cirúrgica       38.495
Obstétrica       9.525
Psiquiátrica     3.760
Name: proportion, dtype: float64

carater_internacao


carater_internacao
Eletiva                55.155
Urgência/Emergência    44.845
Name: proportion, dtype: float64

especialidade_responsavel


especialidade_responsavel
Pneumologia                9.400
Neurologia                 9.325
Clínica Médica             9.255
Ginecologia/Obstetrícia    9.160
Gastroenterologia          9.150
Pediatria                  9.115
Nefrologia                 9.040
Infectologia               8.975
Cardiologia                8.940
Oncologia                  8.870
Ortopedia                  8.770
Name: proportion, dtype: float64

Conclusão: Essa é a distribuição dos dados categóricos.

## Pipeline de tipagem dos dados

* A pipeline aplica técnicas de tipagem das colunas

In [11]:
df_base_01 = pipeline_universal_limpeza(df_base_01)

## Auditoria dos dados

### Idade


In [12]:
adm  = pd.to_datetime(df_base_01["data_admissao"], errors="coerce")
nasc = pd.to_datetime(df_base_01["data_nascimento"], errors="coerce")

idade_aprox = (adm - nasc).dt.days / 365.25

# forçar numérico (se tiver lixo, vira NaN)
idade_aprox = pd.to_numeric(idade_aprox, errors="coerce")

# se quiser idade inteira "aprox" truncada (tipo floor)
df_base_01["idade"] = np.floor(idade_aprox).astype("Int64")


Após análise identificou que cerca de 14657 pacientes estão com idades maiores do que a idade no dia da admissão, optando por uma correção na coluna de idade, trazendo assim a idade no dia da internação.

### Datas


In [13]:
# Data de admissao posterior a data de alta
filtro_data_adm = df_base_01[df_base_01['data_admissao'] > df_base_01['data_alta']]

df_base_01 = df_base_01.drop(filtro_data_adm.index).copy()
df_base_01

,senha_internacao,beneficiario_id,numero_carteirinha,nome_beneficiario,data_nascimento,idade,sexo,uf,municipio,hospital_id,...,uti_flag,suporte_ventilatorio_flag,hemodialise_flag,tempo_autorizacao_horas,auditoria_responsavel,empresa_auditoria,status_regulacao,glosa_flag,valor_total_conta,valor_pago
0,SI20250000001,SI20250000001,CAR405957,P.M.M.,1992-06-17,33,M,SP,Santos,H070,...,Não,Não,Não,30.0,Terceirizada,Empresa Terceira 3,Autorizada,Não,112005.19,112005.19
1,SI20250000002,SI20250000002,CAR658953,V.S.M.,1987-10-24,38,F,MS,Dourados,H013,...,Sim,Não,Sim,22.0,Terceirizada,Empresa Terceira 3,Autorizada com ressalva,Sim,357271.95,303110.50
2,SI20250000003,SI20250000003,CAR547463,N.M.A.,1929-02-16,96,F,MA,Imperatriz,H001,...,Não,Não,Não,26.0,Terceirizada,Empresa Terceira 2,Pendente,Não,32603.64,32603.64
3,SI20250000004,SI20250000004,CAR181581,P.C.N.,1976-10-29,48,M,MG,Juiz de Fora,H043,...,Não,Não,Não,34.0,Terceirizada,Empresa Terceira 2,Autorizada,Sim,24660.48,23423.61
4,SI20250000005,SI20250000005,CAR677468,B.A.R.,1981-02-07,43,M,PA,Belém,H028,...,Não,Não,Sim,21.0,Terceirizada,Empresa Terceira 3,Autorizada,Sim,46771.29,40277.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,SI20250019996,SI20250019996,CAR811091,P.N.C.,1983-05-08,41,F,SP,Santos,H043,...,Não,Não,Não,29.0,Terceirizada,Empresa Terceira 2,Pendente,Não,18658.17,18658.17
19996,SI20250019997,SI20250019997,CAR148487,L.M.T.,1952-11-05,71,M,MA,Imperatriz,H008,...,Não,Sim,Não,27.0,Terceirizada,Empresa Terceira 1,Autorizada,Não,30272.52,30272.52
19997,SI20250019998,SI20250019998,CAR540753,M.F.M.,1995-06-28,30,F,PR,Curitiba,H061,...,Não,Não,Não,13.0,Terceirizada,Empresa Terceira 2,Autorizada,Não,22906.91,22906.91
19998,SI20250019999,SI20250019999,CAR367435,A.M.O.,1948-11-02,76,F,MG,Contagem,H007,...,Não,Não,Não,40.0,Terceirizada,Empresa Terceira 2,Autorizada,Não,12890.38,12890.38


Conclusão:

Encontrados 15 registros de pacientes com admissão posterior a alta.

Ruído > informação :
Esses registros são errados por definição.
Impacto estatístico despresível:0,75% da base

Matém integridade temporal.

In [14]:
# Pacientes sem datas de alta
filtro = df_base_01[df_base_01['data_alta'].isna()]
filtro['motivo_alta']

5        NaN
17       NaN
44       NaN
83       NaN
98       NaN
        ... 
19892    NaN
19928    NaN
19933    NaN
19936    NaN
19987    NaN
Name: motivo_alta, Length: 1200, dtype: category
Categories (2, object): ['Alta por melhora clínica', 'Óbito']

Conclusão:

Existe 1200 registros sem datas de alta e motivos de alta.

Validação lógica: Motivo de alta não pode existir sem data de alta.

Criação de uma coluna: STATUS INTERNAÇÃO

In [16]:
# Interações em abertos:
df_base_01["status_internacao"] = np.where(
    df_base_01["data_alta"].isna(),
    "ATIVA",
    "ENCERRADA"
)

df_base_01["status_internacao"].value_counts()



status_internacao
ENCERRADA    18785
ATIVA         1200
Name: count, dtype: int64

Internações sem data e motivo de alta foram interpretadas como internações ativas,
não sendo tratadas como erro de dados.

### Valores

In [17]:
# Checagem de valores pagos e valores totais de conta negativos
filtro_valores = df_base_01[(df_base_01['valor_pago'] < 0) | (df_base_01['valor_total_conta'] < 0)]
filtro_valores

,senha_internacao,beneficiario_id,numero_carteirinha,nome_beneficiario,data_nascimento,idade,sexo,uf,municipio,hospital_id,...,suporte_ventilatorio_flag,hemodialise_flag,tempo_autorizacao_horas,auditoria_responsavel,empresa_auditoria,status_regulacao,glosa_flag,valor_total_conta,valor_pago,status_internacao


Conclusão: Não há valores negativos

In [18]:
# Tempo de autorização negativos
filtro_tempo_autorizacao = df_base_01[df_base_01['tempo_autorizacao_horas'] < 0]
filtro_tempo_autorizacao[['tempo_autorizacao_horas']]

,tempo_autorizacao_horas


Conclusão : Não há tempo de autorização negativos

In [19]:
delta = (
    df_base_01['data_autorizacao_senha'] -
    df_base_01['data_solicitacao_autorizacao']
)
df_base_01['tempo_autorizacao_horas_new'] = delta.dt.total_seconds() / 3600



filtro  = df_base_01[df_base_01['tempo_autorizacao_horas_new'] != df_base_01['tempo_autorizacao_horas']]
filtro[['tempo_autorizacao_horas','tempo_autorizacao_horas_new']]

,tempo_autorizacao_horas,tempo_autorizacao_horas_new
73,157.0,207.0
1586,6.0,376.0
3559,7.0,88.0
4844,11.0,445.0
5118,7.0,158.0
5274,42.0,281.0
6671,29.0,324.0
6958,37.0,247.0
7022,44.0,318.0
7306,211.0,361.0


Conclusão : Existe 20 registros com tempos de autorização divergentes entre o data_autorizacao_senha e data_solicitacao_autorizacao . Optando por uma nova coluna tempo_autorizacao_horas_new.

### CID - Validação sintática do CID 

In [20]:
# Import da base CID-10-CATEGORIAS.CSV
df_cid = pd.read_csv(r'C:\Projetos\case-ami-saude\data\raw\CID-10-CATEGORIAS.CSV',sep=';', encoding='latin-1')
df_cid

,CAT,CLASSIF,DESCRICAO,DESCRABREV,REFER,EXCLUIDOS,Unnamed: 6
0,A00,NaN,Cólera,A00 Colera,NaN,NaN,NaN
1,A01,NaN,Febres tifóide e paratifóide,A01 Febres tifoide e paratifoide,NaN,NaN,NaN
2,A02,NaN,Outras infecções por Salmonella,A02 Outr infecc p/Salmonella,NaN,NaN,NaN
3,A03,NaN,Shiguelose,A03 Shiguelose,NaN,NaN,NaN
4,A04,NaN,Outras infecções intestinais bacterianas,A04 Outr infecc intestinais bacter,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2040,U80,NaN,Agente resistente à penicilina e antibióticos ...,U80 Agente resist penicilina e antibiót relac,NaN,NaN,NaN
2041,U81,NaN,Agente resistente à vancomicina e antibióticos...,U81 Agente resist vancomicina e antibiót relac,NaN,NaN,NaN
2042,U88,NaN,Agente resistente a múltiplos antibióticos,U88 Agente resistente a múltiplos antibióticos,NaN,NaN,NaN
2043,U89,NaN,Agente resistente a outros antibióticos e a an...,U89 Agente resist outr antibiót e antibiót NE,NaN,NaN,NaN


In [ ]:
# Verificar formato dos CID-10
cid_pattern = re.compile(r"^[A-Z][0-9]{2}$")

df_base_01["cid_principal_formato_ok"] = df_base_01["cid_principal"].str.match(cid_pattern)
df_base_01["cid_secundario_formato_ok"] = df_base_01["cid_secundario"].str.match(cid_pattern)

# Os CID-10 devem seguir o formato letra maiúscula seguida de dois dígitos (ex: A00, B99)
filtro_cid = df_base_01[
    (df_base_01["cid_principal_formato_ok"] == False) |
    (df_base_01["cid_secundario_formato_ok"] == False)
]
filtro_cid[["cid_principal", "cid_secundario", "cid_principal_formato_ok", "cid_secundario_formato_ok"]]



,cid_principal,cid_secundario,cid_principal_formato_ok,cid_secundario_formato_ok


Conclusão: Todos os cids estão no formato esperado: Letra - 2 numeros

In [22]:
# Verificar se o CID está presente na tabela de categorias CID-10
cids_validos = set(df_cid['CAT'])

df_base_01["cid_principal_valido"] = df_base_01["cid_principal"].isin(cids_validos)
df_base_01["cid_secundario_valido"] = (df_base_01["cid_secundario"].isna() | df_base_01["cid_secundario"].isin(cids_validos))

filtro_cid_validos = df_base_01[
    (df_base_01["cid_principal_valido"] == False) |
    (df_base_01["cid_secundario_valido"] == False)
]
filtro_cid_validos[["cid_principal", "cid_secundario", "cid_principal_valido", "cid_secundario_valido"]]

,cid_principal,cid_secundario,cid_principal_valido,cid_secundario_valido


Conclusão: Todos os cids pertencem a uma catégoria existente da base externa CID.

In [23]:
# Verificar se o CID pertence a especialidade médica

mapa_especialidade_cid = {
    "Clínica Médica": (
        "A","B","E","I","J","K","N","R"
    ),
    "Pediatria": (
        "A","B","E","J","P","Q","R","Z"
    ),
    "Oncologia": (
        "C","D"
    ),
    "Nefrologia": (
        "N"
    ),
    "Ginecologia/Obstetrícia": (
        "O","N"
    ),
    "Pneumologia": (
        "J"
    ),
    "Cardiologia": (
        "I"
    ),
    "Gastroenterologia": (
        "K"
    ),
    "Ortopedia": (
        "M","S","T"
    ),
    "Neurologia": (
        "G","I"
    ),
    "Infectologia": (
        "A","B"
    ),
}


df_base_01["cid_principal_compativel_especialidade"] = df_base_01.apply(
    lambda x: cid_compativel_especialidade(
        x["cid_principal"],
        x["especialidade_responsavel"],
        mapa_especialidade_cid
    ),
    axis=1
)

df_base_01["cid_secundario_compativel_especialidade"] = df_base_01.apply(
    lambda x: cid_compativel_especialidade(
        x["cid_secundario"],
        x["especialidade_responsavel"],
        mapa_especialidade_cid
    ),
    axis=1
)

df_base_01['cid_principal_compativel_especialidade'].value_counts()

cid_principal_compativel_especialidade
False    16690
True      3295
Name: count, dtype: int64

In [24]:
df_base_01['cid_secundario_compativel_especialidade'].value_counts()

cid_secundario_compativel_especialidade
True     12541
False     7444
Name: count, dtype: int64

Conclusão: Para futuras auditorias, foi criado uma flag cid_principal_compativel_especialidade para verificar se o cid é correspondente a especialidade médica.

In [25]:
# Cid secundário poderia ser o primario:
cidp = df_base_01["cid_principal"].astype("string").str.strip().str.upper()
cids = df_base_01["cid_secundario"].astype("string").str.strip().str.upper()

tipo = df_base_01["tipo_internacao"].astype("string").str.strip()
carater = df_base_01["carater_internacao"].astype("string").str.strip()
esp = df_base_01["especialidade_responsavel"].astype("string").str.strip()

In [26]:
# Camada 1- 
"""
Tipo de internação Obstétrica → CID principal deve ser capítulo O

Tipo de internação Psiquiátrica → CID principal deve ser capítulo F 
"""

df_base_01["alerta_swap_obstetrica"] = (
    (tipo == "Obstétrica")
    & ~cidp.str.startswith("O", na=False)
    & cids.str.startswith("O", na=False)
)
df_base_01["alerta_swap_psiquiatrica"] = (
    (tipo == "Psiquiátrica")
    & ~cidp.str.startswith("F", na=False)
    & cids.str.startswith("F", na=False)
)


In [65]:
filtro = df_base_01[df_base_01["alerta_swap_psiquiatrica"] ==True]
filtro[['cid_principal','cid_secundario']]

,cid_principal,cid_secundario
257,N18,F32
1149,M17,F32
1323,O80,F32
3203,A41,F32
3800,I21,F32
4886,I10,F32
7713,A41,F32
7925,E11,F32
8909,J44,F32
9273,S82,F32


In [27]:
# Camada 2 - CID secundário mais compatível com a especialidade do que o CID principal

df_base_01["cidp_comp_esp"] = df_base_01.apply(
    lambda x: cid_compativel_especialidade(x["cid_principal"], x["especialidade_responsavel"], mapa_especialidade_cid),
    axis=1
)

df_base_01["cids_comp_esp"] = df_base_01.apply(
    lambda x: cid_compativel_especialidade(x["cid_secundario"], x["especialidade_responsavel"], mapa_especialidade_cid),
    axis=1
)

df_base_01["alerta_swap_especialidade"] = (~df_base_01["cidp_comp_esp"]) & (df_base_01["cids_comp_esp"])


In [28]:
# Camada 3 - CIDs genéricos (capítulos R e Z) não deveriam ser principais se houver um CID específico presente
"""
CIDs dos capítulos R e Z são frequentemente usados como diagnóstico provisório.
Quando eles aparecem como principal e existe um CID secundário mais específico, isso é um padrão clássico de hierarquia invertida.
"""
df_base_01["alerta_swap_generico"] = (
    cidp.str.startswith(("R","Z"), na=False)
    & cids.notna()
    & ~cids.str.startswith(("R","Z"), na=False)
)


In [29]:
# Camada 4 - Para internações eletivas, se o CID principal for de Ortopedia (capítulos S ou T), mas houver um CID secundário compatível com a especialidade, pode ser um swap de CID
"""
Eu não usei caráter como regra dura, mas como priorização.
Por exemplo, trauma como CID principal em internação eletiva é raro. Quando isso acontecia e o secundário era compatível com a especialidade, eu marcava como suspeito.
"""

df_base_01["alerta_swap_carater"] = (
    (carater == "Eletiva")
    & cidp.str.startswith(("S","T"), na=False)
    & df_base_01["cids_comp_esp"]  # secundário faz mais sentido pro setor
)


In [30]:
swap_cols = [
    "alerta_swap_obstetrica",
    "alerta_swap_psiquiatrica",
    "alerta_swap_especialidade",
    "alerta_swap_generico",
    "alerta_swap_carater",
]

df_base_01["alerta_swap"] = df_base_01[swap_cols].any(axis=1)
df_base_01["score_swap"] = df_base_01[swap_cols].sum(axis=1)


In [31]:
# O score de swap é uma métrica que soma o número de alertas acionados para cada internação. Quanto maior o score, mais suspeita é a internação de ter um swap de CID.
""" 
SCORE 0 - Todas as camadas de alerta estão limpas, sem indícios de swap de CID
SCORE 1 - Apenas 1 camada de alerta acionada, suspeita leve de swap
SOCRE 2 - alta probabilidade de inversão
"""

df_base_01['score_swap'].value_counts(normalize=True) *100

score_swap
1    48.996748
0    47.870903
2     3.132349
Name: proportion, dtype: Float64

Conclusão:

* Em uma internação hospitalar:
    * CID principal representa o motivo da internação
    * CID secundário representa comorbidades ou condições associadas
* Na prática, erros comuns acontecem:
    * CID genérico como principal (R/Z)
    * CID compatível com a especialidade ficando como secundário
    * CID obstétrico ou psiquiátrico fora do principal
    * Houve um achado de 3,13% dos cids secundários terem fores indícios de invesão com o principal.
    * Aproximadamente 48% dos cids precisam revisar.

# EDA - BASE 02 | Itens de internação

## Carregamento dos dados

In [32]:
# =========================
# Carregamento da Base 02
# =========================
path_base_02= "C:\\Projetos\\case-ami-saude\\data\\raw\\Base 2 – Itens da Internação.csv"
df_base_02 = pd.read_csv(path_base_02)

df_base_02.shape


(400000, 17)

A base possui 400000 registros e 17 colunas.


## Entendimento estrutural da base

In [33]:
df_base_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   item_id                400000 non-null  object 
 1   senha_internacao       400000 non-null  object 
 2   data_item              400000 non-null  object 
 3   tipo_item              400000 non-null  object 
 4   subtipo_item           400000 non-null  object 
 5   codigo_item            400000 non-null  object 
 6   descricao_item         400000 non-null  object 
 7   quantidade_solicitada  400000 non-null  int64  
 8   quantidade_autorizada  400000 non-null  int64  
 9   unidade_medida         400000 non-null  object 
 10  valor_unitario         400000 non-null  float64
 11  valor_total_item       400000 non-null  float64
 12  setor_execucao         400000 non-null  object 
 13  flag_pacote            400000 non-null  object 
 14  glosa_item_flag        400000 non-nu

In [34]:
df_base_02.describe(include="all")

,item_id,senha_internacao,data_item,tipo_item,subtipo_item,codigo_item,descricao_item,quantidade_solicitada,quantidade_autorizada,unidade_medida,valor_unitario,valor_total_item,setor_execucao,flag_pacote,glosa_item_flag,motivo_glosa,valor_glosado
count,400000,400000,400000,400000,400000,400000,400000,400000.000000,400000.000000,400000,400000.000000,400000.000000,400000,400000,400000,14261,400000.000000
unique,400000,20005,395345,9,21,21,21,NaN,NaN,8,NaN,NaN,7,2,2,7,NaN
top,IT000401837,SI20250001684,2025-01-08 12:10:50,Medicamento,Antibiótico (dose),MD_ATB,Antibiótico (dose),NaN,NaN,un,NaN,NaN,Enfermaria,Não,Não,Técnica - item não pertinente ao CID,NaN
freq,1,56,3,116441,58559,58559,58559,NaN,NaN,121337,NaN,NaN,157096,390513,385739,2438,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.117392,9.728160,NaN,1105.629264,2596.045271,NaN,NaN,NaN,NaN,70.249906
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.222409,13.956731,NaN,2958.618241,4161.837195,NaN,NaN,NaN,NaN,667.575420
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,NaN,9.830000,0.000000,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,96.540000,523.757500,NaN,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,3.000000,NaN,207.210000,1270.120000,NaN,NaN,NaN,NaN,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.000000,10.000000,NaN,907.950000,2896.830000,NaN,NaN,NaN,NaN,0.000000


Conclusão – Estrutura

A base apresenta estrutura compatível com granularidade por itens.

## Qualidade básica dos dados

In [35]:
# Valores ausentes
df_base_02.isna().mean().sort_values(ascending=False) * 100


motivo_glosa             96.43475
item_id                   0.00000
senha_internacao          0.00000
tipo_item                 0.00000
data_item                 0.00000
codigo_item               0.00000
descricao_item            0.00000
quantidade_solicitada     0.00000
subtipo_item              0.00000
quantidade_autorizada     0.00000
unidade_medida            0.00000
valor_total_item          0.00000
valor_unitario            0.00000
setor_execucao            0.00000
flag_pacote               0.00000
glosa_item_flag           0.00000
valor_glosado             0.00000
dtype: float64

Conclusão: 96% dos dados de motivo glosa tem valores faltantes

In [36]:
# Valores duplicados
df_base_02.duplicated().sum()


np.int64(0)

Conclusão : Não foram encontrados valores duplicados.

In [37]:
df_base_02['subtipo_item']

0           Exames de imagem (un)
1                 Honorários (un)
2          Procedimento Cirúrgico
3               Diária Enfermaria
4             Medicamentos (dose)
                   ...           
399995          Diária Enfermaria
399996         Antibiótico (dose)
399997     Taxas e materiais (un)
399998    Fonoaudiologia (sessão)
399999    Fonoaudiologia (sessão)
Name: subtipo_item, Length: 400000, dtype: object

In [38]:
df_base_02.columns

Index(['item_id', 'senha_internacao', 'data_item', 'tipo_item', 'subtipo_item',
       'codigo_item', 'descricao_item', 'quantidade_solicitada',
       'quantidade_autorizada', 'unidade_medida', 'valor_unitario',
       'valor_total_item', 'setor_execucao', 'flag_pacote', 'glosa_item_flag',
       'motivo_glosa', 'valor_glosado'],
      dtype='object')

In [39]:
# Dominio de valores categoricos
for col in ["tipo_item", "subtipo_item", "descricao_item","unidade_medida","setor_execucao"]:
    print(col)
    display(df_base_02[col].value_counts(dropna=False, normalize=True) * 100)


tipo_item


tipo_item
Medicamento        29.11025
Terapia seriada    25.01625
Exame              15.48525
Taxa               13.98700
Procedimento        8.18975
Diária              5.99525
Suporte             1.17575
OPME                0.59000
Terapia             0.45050
Name: proportion, dtype: float64

subtipo_item


subtipo_item
Antibiótico (dose)               14.63975
Medicamentos (dose)              14.47050
Fonoaudiologia (sessão)           8.38450
Nutrição (atendimento)            8.32175
Fisioterapia (sessão)             8.31000
Exames de imagem (un)             7.75175
Exames laboratoriais (pacote)     7.73100
Honorários (un)                   7.01975
Taxas e materiais (un)            6.96725
Diária Enfermaria                 4.98150
Procedimento Cirúrgico            4.30650
Procedimento Clínico              2.30900
Procedimento SADT                 1.57400
Ventilação mecânica (dia)         1.17575
Diária UTI                        1.01375
Hemodiálise (sessão)              0.45050
OPME Geral (material)             0.36675
OPME Ortopedia (kit)              0.18400
OPME Cardiologia (stent)          0.03925
Imagem                            0.00250
Cirurgia obstétrica               0.00025
Name: proportion, dtype: float64

descricao_item


descricao_item
Antibiótico (dose)                14.63975
Medicamentos (dose)               14.47050
Fonoaudiologia (sessão)            8.38450
Nutrição (atendimento)             8.32175
Fisioterapia (sessão)              8.31000
Exames de imagem (un)              7.75175
Exames laboratoriais (pacote)      7.73100
Honorários (un)                    7.01975
Taxas e materiais (un)             6.96725
Diária Enfermaria                  4.98150
Procedimento Cirúrgico             4.30650
Procedimento Clínico               2.30900
Procedimento SADT                  1.57400
Ventilação mecânica (dia)          1.17575
Diária UTI                         1.01375
Hemodiálise (sessão)               0.45050
OPME Geral (material)              0.36675
OPME Ortopedia (kit)               0.18400
OPME Cardiologia (stent)           0.03925
Exame não vinculado (anomalia)     0.00250
Parto cesáreo                      0.00025
Name: proportion, dtype: float64

unidade_medida


unidade_medida
un              30.33425
dose            29.11025
sessão          25.46675
pacote           7.73100
diária           7.17100
kit              0.18400
exame            0.00250
procedimento     0.00025
Name: proportion, dtype: float64

setor_execucao


setor_execucao
Enfermaria          39.27400
Farmácia            29.11025
SADT                17.05675
Administrativo       7.01975
Centro Cirúrgico     4.89675
UTI                  2.64000
Diagnóstico          0.00250
Name: proportion, dtype: float64

Conclusão: Essa é a distribuição dos dados categóricos.

## Pipeline de tipagem dos dados

* A pipeline aplica técnicas de tipagem das colunas

In [40]:
df_base_02 = pipeline_universal_limpeza(df_base_02)

## Auditoria dos dados

In [41]:
df_base_02.columns

Index(['item_id', 'senha_internacao', 'data_item', 'tipo_item', 'subtipo_item',
       'codigo_item', 'descricao_item', 'quantidade_solicitada',
       'quantidade_autorizada', 'unidade_medida', 'valor_unitario',
       'valor_total_item', 'setor_execucao', 'flag_pacote', 'glosa_item_flag',
       'motivo_glosa', 'valor_glosado'],
      dtype='object')

### Valores

In [42]:
filtro = df_base_02[df_base_02['quantidade_autorizada'] > df_base_02['quantidade_solicitada']]
filtro

,item_id,senha_internacao,data_item,tipo_item,subtipo_item,codigo_item,descricao_item,quantidade_solicitada,quantidade_autorizada,unidade_medida,valor_unitario,valor_total_item,setor_execucao,flag_pacote,glosa_item_flag,motivo_glosa,valor_glosado


Conclusão: Não há quantidade autorizada maior que quantidade solicitada.

In [43]:
# Descobrir quais itens tem mais de um valor diferente

problemas = (
    df_base_02
    .groupby("codigo_item")["valor_unitario"]
    .nunique()
    .reset_index(name="qtd_valores_distintos")
    .sort_values(by="qtd_valores_distintos", ascending=False)
)

problemas = problemas[problemas["qtd_valores_distintos"] > 1]
problemas


,codigo_item,qtd_valores_distintos
2,EX_IMG,28369
3,EX_LAB,26843
19,TX_HON,26233
17,TS_FON,20647
18,TS_NUT,19472
20,TX_MAT,19191
6,MD_ATB,19001
0,DI_ENF,18697
16,TS_FIS,18233
12,PR_CIR,17069


Conclusão : Muitos itens tem valores diferentes

In [44]:
df_base_02["valor_total_new"] = (
    df_base_02["quantidade_autorizada"] * df_base_02["valor_unitario"]
).round(2)

mask_diff = ~np.isclose(
    df_base_02["valor_total_new"],
    df_base_02["valor_total_item"],
    atol=0.05,
)

filtro = df_base_02.loc[mask_diff, ["quantidade_autorizada","valor_unitario","valor_total_item","valor_total_new"]]
filtro


,quantidade_autorizada,valor_unitario,valor_total_item,valor_total_new
83537,30,107.86,3235.68,3235.80
118312,27,82.84,2236.55,2236.68
135101,48,84.76,4068.29,4068.48
219029,22,151.54,3333.79,3333.88
277239,33,117.68,3883.57,3883.44
280695,29,178.00,5161.88,5162.00
386221,29,189.08,5483.44,5483.32
391878,0,2243.12,1.26,0.00


In [45]:
diferenca = sum(df_base_02["valor_total_item"])  - sum(df_base_02["valor_total_new"])
diferenca

-0.14000022411346436

Conclusão : Foi identificado, alguns valores totais diferentes dos valores totais ba base, resultando uma diferença total de : -0,14 centavos, o que não é muito relevante, mas precisa ter regras de arredondamentos.

In [46]:
# Verificados glosas

df_base_02[
    df_base_02["valor_glosado"] > df_base_02["valor_total_item"]
]

,item_id,senha_internacao,data_item,tipo_item,subtipo_item,codigo_item,descricao_item,quantidade_solicitada,quantidade_autorizada,unidade_medida,valor_unitario,valor_total_item,setor_execucao,flag_pacote,glosa_item_flag,motivo_glosa,valor_glosado,valor_total_new
245,IT000283728,SI20250014195,2025-02-16 01:05:49,Medicamento,Medicamentos (dose),MD_GER,Medicamentos (dose),27,13,dose,44.57,579.41,Farmácia,Não,Sim,Técnica - item não pertinente ao CID,1203.39,579.41
968,IT000356109,SI20250017831,2025-07-15 06:43:25,Medicamento,Medicamentos (dose),MD_GER,Medicamentos (dose),28,20,dose,51.39,1027.80,Farmácia,Não,Sim,Técnica - divergência de diretriz,1336.14,1027.80
1684,IT000238932,SI20250011955,2024-12-09 21:23:36,Exame,Exames laboratoriais (pacote),EX_LAB,Exames laboratoriais (pacote),2,1,pacote,602.37,602.37,SADT,Não,Sim,Técnica - item não pertinente ao CID,1204.74,602.37
3148,IT000174273,SI20250008721,2025-12-10 21:55:20,Medicamento,Medicamentos (dose),MD_GER,Medicamentos (dose),12,11,dose,103.86,1142.46,Farmácia,Não,Sim,Técnica - divergência de diretriz,1246.32,1142.46
3781,IT000320027,SI20250016028,2024-11-20 03:25:37,Taxa,Taxas e materiais (un),TX_MAT,Taxas e materiais (un),1,0,un,143.48,0.00,Enfermaria,Não,Sim,Administrativa - cobrança em duplicidade,143.48,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399011,IT000400865,SI20250003820,2025-11-29 02:48:03,Medicamento,Antibiótico (dose),MD_ATB,Antibiótico (dose),5,0,dose,144.03,0.00,Farmácia,Não,Sim,Administrativa - código incompatível,613.41,0.00
399046,IT000400900,SI20250007646,2025-09-11 04:55:46,Diária,Diária Enfermaria,DI_ENF,Diária Enfermaria,12,12,diária,865.13,10381.56,Enfermaria,Não,Sim,Técnica - item não pertinente ao CID,10381.60,10381.56
399239,IT000401093,SI20250013270,2025-01-31 17:14:50,Taxa,Taxas e materiais (un),TX_MAT,Taxas e materiais (un),2,0,un,455.72,0.00,Enfermaria,Não,Sim,Técnica - divergência de diretriz,911.44,0.00
399718,IT000401572,SI20250012283,2025-06-28 11:43:59,Procedimento,Procedimento Clínico,PR_CLI,Procedimento Clínico,1,0,un,1083.99,0.00,Enfermaria,Não,Sim,Administrativa - cobrança em duplicidade,1009.50,0.00


Conclusão: Foi encontrado 817 registros que o valor glosado é maior que o valor total do item, identificando um possível erro.

In [47]:
# Glosa de quantidade acima do padrão, onde o valor glosado é igual ao valor total do item
df_base_02.query(
    "motivo_glosa == 'Técnica - quantidade acima do padrão' and "
    "valor_glosado == valor_total_item"
)


,item_id,senha_internacao,data_item,tipo_item,subtipo_item,codigo_item,descricao_item,quantidade_solicitada,quantidade_autorizada,unidade_medida,valor_unitario,valor_total_item,setor_execucao,flag_pacote,glosa_item_flag,motivo_glosa,valor_glosado,valor_total_new
279,IT000314673,SI20250015750,2025-02-10 20:54:42,Exame,Exames laboratoriais (pacote),EX_LAB,Exames laboratoriais (pacote),2,2,pacote,377.90,755.80,SADT,Não,Sim,Técnica - quantidade acima do padrão,755.80,755.80
678,IT000210476,SI20250010519,2025-08-03 18:01:11,Procedimento,Procedimento Clínico,PR_CLI,Procedimento Clínico,1,1,un,842.05,842.05,Enfermaria,Não,Sim,Técnica - quantidade acima do padrão,842.05,842.05
704,IT000162855,SI20250008132,2024-09-11 06:42:38,Taxa,Honorários (un),TX_HON,Honorários (un),1,1,un,1367.00,1367.00,Administrativo,Não,Sim,Técnica - quantidade acima do padrão,1367.00,1367.00
1736,IT000161503,SI20250008069,2025-04-16 15:35:12,Taxa,Honorários (un),TX_HON,Honorários (un),2,2,un,1022.93,2045.86,Administrativo,Não,Sim,Técnica - quantidade acima do padrão,2045.86,2045.86
1875,IT000075523,SI20250003796,2025-10-06 13:38:35,Procedimento,Procedimento SADT,PR_SADT,Procedimento SADT,1,1,un,2086.49,2086.49,SADT,Não,Sim,Técnica - quantidade acima do padrão,2086.49,2086.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397779,IT000372814,SI20250018661,2024-12-26 08:56:49,Medicamento,Antibiótico (dose),MD_ATB,Antibiótico (dose),30,30,dose,60.21,1806.30,Farmácia,Não,Sim,Técnica - quantidade acima do padrão,1806.30,1806.30
398571,IT000400425,SI20250009286,2025-09-02 12:47:25,Taxa,Taxas e materiais (un),TX_MAT,Taxas e materiais (un),2,2,un,181.62,363.24,Enfermaria,Não,Sim,Técnica - quantidade acima do padrão,363.24,363.24
398743,IT000400597,SI20250019540,2025-01-31 01:19:43,Taxa,Taxas e materiais (un),TX_MAT,Taxas e materiais (un),3,3,un,121.83,365.49,Enfermaria,Não,Sim,Técnica - quantidade acima do padrão,365.49,365.49
399293,IT000401147,SI20250003199,2024-08-03 09:37:47,Medicamento,Antibiótico (dose),MD_ATB,Antibiótico (dose),43,43,dose,90.50,3891.50,Farmácia,Não,Sim,Técnica - quantidade acima do padrão,3891.50,3891.50


Conclusão: esse ponto merece uma revisão, pois o valor glosado está igual ao valor total do item e o motivo da glosa é quantidade acima do padrão, quantidade encontrada 1248 registros. O valor glosado deveria ser menor que o valor total.

In [48]:
# Glosa Parcial onde deveria ser glosa total
df_base_02.query(
    "motivo_glosa == 'Administrativa - cobrança em duplicidade' and "
    "valor_glosado < valor_total_item"
)



,item_id,senha_internacao,data_item,tipo_item,subtipo_item,codigo_item,descricao_item,quantidade_solicitada,quantidade_autorizada,unidade_medida,valor_unitario,valor_total_item,setor_execucao,flag_pacote,glosa_item_flag,motivo_glosa,valor_glosado,valor_total_new
1336,IT000223747,SI20250011187,2025-05-08 04:29:41,Taxa,Honorários (un),TX_HON,Honorários (un),4,4,un,920.70,3682.80,Administrativo,Não,Sim,Administrativa - cobrança em duplicidade,1485.44,3682.80
1513,IT000035016,SI20250001756,2025-02-19 18:17:35,Taxa,Taxas e materiais (un),TX_MAT,Taxas e materiais (un),3,3,un,151.60,454.80,Enfermaria,Não,Sim,Administrativa - cobrança em duplicidade,26.57,454.80
1521,IT000276679,SI20250013840,2024-09-23 06:27:08,Terapia seriada,Fisioterapia (sessão),TS_FIS,Fisioterapia (sessão),11,11,sessão,102.68,1129.48,Enfermaria,Não,Sim,Administrativa - cobrança em duplicidade,1039.64,1129.48
1686,IT000286806,SI20250014343,2024-10-31 05:01:45,Terapia seriada,Fonoaudiologia (sessão),TS_FON,Fonoaudiologia (sessão),10,10,sessão,172.50,1725.00,Enfermaria,Não,Sim,Administrativa - cobrança em duplicidade,1303.44,1725.00
1860,IT000112731,SI20250005651,2025-10-27 08:14:51,Suporte,Ventilação mecânica (dia),SV_VM,Ventilação mecânica (dia),1,1,diária,1588.37,1588.37,UTI,Não,Sim,Administrativa - cobrança em duplicidade,851.73,1588.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399187,IT000401041,SI20250018331,2024-12-04 10:09:49,Procedimento,Procedimento Cirúrgico,PR_CIR,Procedimento Cirúrgico,1,1,un,5715.74,5715.74,Centro Cirúrgico,Não,Sim,Administrativa - cobrança em duplicidade,4443.05,5715.74
399404,IT000401258,SI20250003011,2025-06-15 13:14:40,Diária,Diária Enfermaria,DI_ENF,Diária Enfermaria,4,4,diária,1685.60,6742.40,Enfermaria,Não,Sim,Administrativa - cobrança em duplicidade,4810.49,6742.40
399444,IT000401298,SI20250002923,2024-10-19 04:00:50,Exame,Exames laboratoriais (pacote),EX_LAB,Exames laboratoriais (pacote),1,1,pacote,308.32,308.32,SADT,Não,Sim,Administrativa - cobrança em duplicidade,52.58,308.32
399681,IT000401535,SI20250015996,2024-08-02 15:18:57,Procedimento,Procedimento Cirúrgico,PR_CIR,Procedimento Cirúrgico,1,1,un,8894.12,8894.12,Centro Cirúrgico,Não,Sim,Administrativa - cobrança em duplicidade,3282.81,8894.12


Conclusão: A glosa aqui deveria ser igual ao valor total do item, pois trata-se de item duplicado.

# EDA - Join Base 1 e base 2

In [49]:
df_base_01_new = pd.read_csv(path_base_01)
df_base_02_new = pd.read_csv(path_base_02)

## Intercessão de colunas

In [50]:
colunas_iguais = set(df_base_01.columns).intersection(set(df_base_02.columns))
colunas_iguais

{'senha_internacao'}

Conclusão: Só existe uma coluna para merge entre as duas tabelas; {'senha_internacao'}.

## Qualidade senha internação

In [59]:
internacao_nao_encontradas = df_base_02_new[~df_base_02_new['senha_internacao'].isin(df_base_01_new ['senha_internacao'])]
internacao_nao_encontradas['senha_internacao'].nunique()

10

Conclusão: Foram encontradas 10 {'senha_internacao'} que estão presentes em uma base e não estão na outra.


## Cruzamento para visão consolidada

In [52]:
# Agrupando a Base 02 por senha para cruzar com a Base 01
base_02_agg = df_base_02_new.groupby('senha_internacao').agg({
    'valor_total_item': 'sum',
    'valor_glosado': 'sum',
    'item_id': 'count'
}).rename(columns={'item_id': 'qtd_itens'}).reset_index()

# Merge Final
df_consolidado = pd.merge(df_base_01_new, base_02_agg, on='senha_internacao', how='left')

In [54]:
df_consolidado

,senha_internacao,beneficiario_id,numero_carteirinha,nome_beneficiario,data_nascimento,idade,sexo,uf,municipio,hospital_id,...,tempo_autorizacao_horas,auditoria_responsavel,empresa_auditoria,status_regulacao,glosa_flag,valor_total_conta,valor_pago,valor_total_item,valor_glosado,qtd_itens
0,SI20250000001,SI20250000001,CAR405957,P.M.M.,1992-06-17 00:00:00,33,M,SP,Santos,H070,...,30.0,Terceirizada,Empresa Terceira 3,Autorizada,Não,112005.19,112005.19,105888.66,0.00,20.0
1,SI20250000002,SI20250000002,CAR658953,V.S.M.,1987-10-24 00:00:00,38,F,MS,Dourados,H013,...,22.0,Terceirizada,Empresa Terceira 3,Autorizada com ressalva,Sim,357271.95,303110.50,303439.62,54161.45,27.0
2,SI20250000003,SI20250000003,CAR547463,N.M.A.,1929-02-16 00:00:00,96,F,MA,Imperatriz,H001,...,26.0,Terceirizada,Empresa Terceira 2,Pendente,Não,32603.64,32603.64,35417.69,0.00,13.0
3,SI20250000004,SI20250000004,CAR181581,P.C.N.,1976-10-29 00:00:00,49,M,MG,Juiz de Fora,H043,...,34.0,Terceirizada,Empresa Terceira 2,Autorizada,Sim,24660.48,23423.61,24660.48,1236.87,18.0
4,SI20250000005,SI20250000005,CAR677468,B.A.R.,1981-02-07 00:00:00,45,M,PA,Belém,H028,...,21.0,Terceirizada,Empresa Terceira 3,Autorizada,Sim,46771.29,40277.38,39977.75,6493.91,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,SI20250019996,SI20250019996,CAR811091,P.N.C.,1983-05-08 00:00:00,42,F,SP,Santos,H043,...,29.0,Terceirizada,Empresa Terceira 2,Pendente,Não,18658.17,18658.17,18658.15,0.00,22.0
19996,SI20250019997,SI20250019997,CAR148487,L.M.T.,1952-11-05 00:00:00,73,M,MA,Imperatriz,H008,...,27.0,Terceirizada,Empresa Terceira 1,Autorizada,Não,30272.52,30272.52,23871.19,0.00,12.0
19997,SI20250019998,SI20250019998,CAR540753,M.F.M.,1995-06-28 00:00:00,30,F,PR,Curitiba,H061,...,13.0,Terceirizada,Empresa Terceira 2,Autorizada,Não,22906.91,22906.91,23942.85,0.00,18.0
19998,SI20250019999,SI20250019999,CAR367435,A.M.O.,1948-11-02 00:00:00,77,F,MG,Contagem,H007,...,40.0,Terceirizada,Empresa Terceira 2,Autorizada,Não,12890.38,12890.38,13232.60,0.00,12.0


## Teste de hipótese


In [ ]:
df_consolidado['uti_flag'] = df_consolidado['uti_flag'].astype(str).str.strip().str.upper()


uti_sim = df_consolidado[df_consolidado['uti_flag'] == 'SIM']['valor_total_conta'].dropna()
uti_nao = df_consolidado[df_consolidado['uti_flag'] == 'NÃO']['valor_total_conta'].dropna()


t_stat, p_val = stats.ttest_ind(uti_sim, uti_nao, equal_var=False) 

print(f"Média Custo com UTI: R$ {uti_sim.mean():.2f}")
print(f"Média Custo sem UTI: R$ {uti_nao.mean():.2f}")
print(f"P-valor: {p_val:.10f}")

Média Custo com UTI: R$ 76419.87
Média Custo sem UTI: R$ 46318.54
P-valor: 0.0000000000


Conclusão: A análise revelou que o custo médio das internações com UTI é significativamente maior do que o custo médio das internações sem UTI, com um p-valor extremamente baixo, indicando uma diferença estatisticamente significativa entre os dois grupos.